In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing

In [2]:
from quantopian.pipeline.filters import Q1500US

In [3]:
universe = Q1500US()

In [4]:
from quantopian.pipeline.data import morningstar

In [5]:
sector = morningstar.asset_classification.morningstar_sector_code.latest

In [6]:
energy_sector = sector.eq(309)

In [7]:
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

In [8]:
dollar_volume = AverageDollarVolume(window_length = 30)

In [9]:
high_dollar_volume = dollar_volume.percentile_between(90,100)

In [10]:
top_open_prices = USEquityPricing.open.latest.top(50,mask=high_dollar_volume)

In [11]:
high_close_price = USEquityPricing.close.latest.percentile_between(9,100,mask = top_open_prices)

In [12]:
def make_pipeline():
    
    # Universe Q1500US
    base_universe = Q1500US()
    # Energy Sector
    energy_secotor = sector.eq(309)
    
    # Make Mask of 1500US and Energy
    base_energy = base_universe & energy_sector
    
    # Dollar Volume (30 days) Grab the Info
    dollar_volume = AverageDollarVolume(window_length=30)
    
    # Grab the top 5% in avg dollar volume
    high_dollar_volume = dollar_volume.percentile_between(95,100)
    
    # combine filters
    top_five_base_energy = base_energy & high_dollar_volume
    # 10 day mean close
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length = 10, mask = top_five_base_energy) 
    # 30 day mean close
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length = 30, mask = top_five_base_energy) 
    # percent difference
    percent_difference = (mean_10-mean_30)/mean_30
    # list of shorts
    shorts = percent_difference < 0 
    # list of longs
    longs = percent_difference > 0
    # final mask/filter for anything in shorts or longs
    securities_to_trade = (shorts | longs)
    # return pipeline
    return Pipeline(columns={
        'longs': longs,
        'shorts': shorts,
        'perc_diff': percent_difference,
    }, screen=securities_to_trade)

In [13]:
result = run_pipeline(make_pipeline(),'2020-09-21','2020-09-21')
result

longs  perc_diff shorts
2020-09-21 00:00:00+00:00 Equity(2564 [EOG])   False  -0.085366   True
                          Equity(3443 [HAL])   False  -0.058647   True
                          Equity(5035 [MRO])   False  -0.130577   True
                          Equity(5213 [NBL])   False  -0.062324   True
                          Equity(5729 [OXY])   False  -0.140149   True
                          Equity(6928 [SLB])   False  -0.028229   True
                          Equity(7990 [VLO])   False  -0.072782   True
                          Equity(8214 [WMB])   False  -0.010393   True
                          Equity(8347 [XOM])   False  -0.067783   True
                          Equity(23112 [CVX])  False  -0.060684   True
                          Equity(23998 [COP])  False  -0.078507   True
                          Equity(40852 [KMI])  False  -0.047815   True
                          Equity(41636 [MPC])  False  -0.083416   True
                          Equity(42788 [PSX])  False  -0.032906   True